In [56]:
import pandas as pd
import numpy as np
import requests
from requests.exceptions import HTTPError
import time

In [57]:
def em(samples, thetas):
    
    ll_old = 0
    max_iter=100     # maximum iteration for thetas to conclude in EM algorithm 
    tol=1e-6         # comparing the old and new thetas values till the accuracy of 0.000001
    
    
    for i in range(max_iter):

        ll_new = 0
        coinA = []
        coinB = []

        # E-step: calculate probability distributions
        for x in samples:
            # Calculate the probability of coin A and coin B given different draws. 
            # P(draw1|coinA), P(draw1|coinB) for all draws. Use condiitonal probability and product rule to get it.
            # p(draw|coin) = p(draw,coin)/p(coin)
            
            num_A = (np.power(thetas[0][0], x[0]) * np.power(thetas[0][1], x[1]))
            den_A = num_A + (np.power(thetas[1][0], x[0]) * np.power(thetas[1][1], x[1]))
            prob_A = num_A / den_A
           
            
            num_B = (np.power(thetas[1][0], x[0]) * np.power(thetas[1][1], x[1]))
            prob_B = 1 - prob_A  # sum of probabilty of any draw given coin A and coin B shoule be 1.
            
            #calculate number of (heads, tails) from coin A and coin B for each draws.
            numHT_A = (prob_A * x[0] , prob_A * x[1])
            numHT_B = (prob_B * x[0] , prob_B * x[1])
            
            # Store number of (heads, tails) over draws from different coin in separate list. It will be used M-step
            coinA.append(numHT_A)
            coinB.append(numHT_B)
            
            # update complete log likelihood
            ll_new += prob_A * num_A + prob_B * num_B
        
        
        # Calculate total number of heads and tails from coin A after all the draws
        tol_heads_tails_A = np.sum(coinA, axis=0)
    
        # M-step : calculate new thetas by (#heads/(#heads + #tails))
        thetas[0][0] = tol_heads_tails_A[0] / (tol_heads_tails_A[0] + tol_heads_tails_A[1])
        thetas[0][1] = 1 - thetas[0][0]
        
        # Calculate total number of heads and tails from coin B after all the draws
        tol_heads_tails_B = np.sum(coinB, axis=0)
        thetas[1][0] = tol_heads_tails_B[0] / (tol_heads_tails_B[0] + tol_heads_tails_B[1])
        thetas[1][1] = 1 - thetas[1][0]
        
        print("New thetas values after iteration : ", i+1, thetas[0], thetas[1])
        
        # Check log likelihood to conclude on convergences of thetas.
        if np.abs(ll_new - ll_old) < tol:
            print("breaking when i = ", i)
            break
        ll_old = ll_new
        
    return thetas    


In [58]:
time1 = time.time()
samples = []
# Code to get 30 draws of 20 coin flips from the given API.
for i in range(30):
    try:
        response = requests.get(' https://24zl01u3ff.execute-api.us-west-1.amazonaws.com/beta')
        response.raise_for_status()
        # access JSOn content
        jsonResponse = response.json()
        #print("Entire JSON response")
        #print(jsonResponse)
        #print(jsonResponse["body"])
        flips = jsonResponse["body"]
        sample = (flips.count('1'), flips.count('0'))
        #print(sample)
        samples.append(sample)
        #q.put(sample)
    

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
 
time2 = time.time()
print(f'Took {time2-time1:.2f} s')
print(samples)

Took 34.81 s
[(6, 14), (7, 13), (8, 12), (4, 16), (7, 13), (8, 12), (14, 6), (12, 8), (12, 8), (4, 16), (15, 5), (7, 13), (14, 6), (14, 6), (17, 3), (16, 4), (4, 16), (6, 14), (12, 8), (17, 3), (13, 7), (14, 6), (15, 5), (4, 16), (6, 14), (11, 9), (4, 16), (7, 13), (13, 7), (10, 10)]


In [48]:

#L = []
def dothing(L, i):  # the managed list `L` passed explicitly.
    L.append("anything")

if __name__ == "__main__":
    with Manager() as manager:
        L = manager.list()  # <-- can be shared between processes.
        processes = []
        for i in range(1):
            p = Process(target=get_samples, args=(L,i))  # Passing the list
            p.start()
            processes.append(p)
            
        for p in processes:
            p.join()
        
        print(L)
#print("Outside of IF")
#print(L)    

[]


In [ ]:
if __name__ == "__main__":
    q = Queue()
    processes = []
    for i in range(1):
        p = Process(target=get_samples, args=(q,i))
        p.start()
        processes.append(p)
            
    for p in processes:
        print(q.get())
        p.join()
        
    
        
    

In [81]:
import pandas as pd
import concurrent.futures
import requests
import time
import os

out = []
CONNECTIONS = 20 #100
TIMEOUT = 5

url = 'https://24zl01u3ff.execute-api.us-west-1.amazonaws.com/beta'

def load_url(url):
    print("load_url pid:", os.getpid())
    response = requests.get(url)
    response.raise_for_status()
    jsonResponse = response.json()
    flips = jsonResponse["body"]
    sample = (flips.count('1'), flips.count('0'))
    return sample

time1 = time.time()
# In case of IO operations use Thread (one GIL), in case of data crunching use Process(separate GIL)
with concurrent.futures.ProcessPoolExecutor(max_workers=CONNECTIONS) as executor: 
#with concurrent.futures.ProcessPoolExecutor(max_workers=CONNECTIONS) as executor:
    future_to_url = (executor.submit(load_url, url) for _ in range(30))
    
    for future in concurrent.futures.as_completed(future_to_url):
        try:
            data = future.result()
        except Exception as exc:
            data = str(type(exc))
        finally:
            out.append(data)

            #print(str(len(out)),end="\r")

    
time2 = time.time()
print(f'Took {time2-time1:.2f} s')
#print(pd.Series(out))
print(out)

load_url pid: 3671
load_url pid: 3675
load_url pid: 3669
load_url pid: 3668
load_url pid: 3673
load_url pid: 3672
load_url pid: 3676
load_url pid: 3670
load_url pid: 3674
load_url pid: 3677
load_url pid: 3678
load_url pid: 3681
load_url pid: 3680
load_url pid: 3679
load_url pid: 3682
load_url pid: 3683
load_url pid: 3685
load_url pid: 3686
load_url pid: 3684
load_url pid: 3687
load_url pid: 3672
load_url pid: 3673
load_url pid: 3670
load_url pid: 3676
load_url pid: 3671
load_url pid: 3675
load_url pid: 3679
load_url pid: 3669
load_url pid: 3680
load_url pid: 3678
Took 2.59 s
[(4, 16), (16, 4), (3, 17), (13, 7), (16, 4), (3, 17), (13, 7), (17, 3), (2, 18), (6, 14), (5, 15), (10, 10), (5, 15), (15, 5), (8, 12), (3, 17), (15, 5), (16, 4), (15, 5), (7, 13), (18, 2), (13, 7), (6, 14), (9, 11), (14, 6), (2, 18), (15, 5), (17, 3), (10, 10), (12, 8)]
